Building likelihood maps for a set of representative images.

> TODO: use images from dataset and superimpose the bounding boxes on them


In [ ]:
import retinoto_py as fovea
subset_factor = 50
args = fovea.Params(do_fovea=True, batch_size=1, subset_factor=subset_factor)
args

In [ ]:
dataset = 'bbox'
VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
val_dataset = fovea.get_dataset(args, VAL_DATA_DIR, do_full_preprocess=False)
val_loader = fovea.get_loader(args, val_dataset)


In [ ]:
model_filename = args.data_cache /  f'32_fovea_model_name={args.model_name}_dataset={dataset}.pth'
model = fovea.load_model(args, model_filename=model_filename)
model_filename

In [ ]:
from torchvision.transforms import v2 as transforms
npy_filename = args.data_cache / '42_likelihood_maps.npy'

# %rm {npy_filename}  # FORCING RECOMPUTE

In [ ]:
resolution = (100, 100)
resolution = (30, 30)
resolution = (20, 20)
size_ratio = .4

if npy_filename.exists():
    results = fovea.np.load(npy_filename)
else:
    n_dataset = len(val_dataset)
    with fovea.torch.no_grad():
        results = fovea.np.empty((resolution[0], resolution[1], n_dataset))
        for i_batch, (image, true_idx) in fovea.tqdm(enumerate(val_loader), total=n_dataset):
            image, true_idx = image.to(args.device), true_idx.to(args.device)

            pos_H, pos_W, probas = fovea.compute_likelihood_map(args, model, image.squeeze(0), size_ratio=size_ratio, resolution=resolution)
            results[:, :, i_batch] = probas[:, true_idx].cpu().numpy().reshape(resolution)
        fovea.np.save(npy_filename, results)        

In [ ]:
# fig, ax = fovea.plt.subplots()
# contour = ax.contourf(results.mean(axis=-1))
# fig.colorbar(contour, ax=ax)  # Add colorbar
# ax.axis('square')
# fig.set_facecolor(color='white')


In [ ]:
results.shape

In [ ]:
i_image = 2
proba = results[:, :, i_image]
proba.min(), proba.max()

In [ ]:
fig, ax = fovea.plt.subplots()
contour = ax.contourf(results.mean(axis=-1))
fig.colorbar(contour, ax=ax)  # Add colorbar
ax.axis('square')
fig.set_facecolor(color='white')


In [ ]:
import pandas as pd
import numpy as np

pos_H, pos_W = fovea.get_positions(1, 1, resolution=resolution)
pos_H.min(), pos_H.max(), pos_W.min(), pos_W.max(), pos_H.shape, results.shape

In [ ]:
data = []
for do_fovea in [True, False]:    
    results = fovea.np.load(npy_filename)
    data_= pd.DataFrame({
        'horizontal position': pos_H,
        'vertical position': pos_W,
        'probas': results.mean(axis=-1).T.flatten(),
        'retinotopy': 'foveated' if do_fovea else 'uniform' 
    })
    data.append(data_)
combined_data = pd.concat(data, ignore_index=True)
combined_data

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter

def visualize_likelihood_map(likelihood_map, pos_H, pos_W):

    # Lisser la heatmap avec un filtre gaussien
    likelihood_map_smoothed = gaussian_filter(likelihood_map, sigma=3)

    # Calcul des probabilités marginales
    marginal_H = likelihood_map_smoothed.sum(axis=1)
    marginal_W = likelihood_map_smoothed.sum(axis=0)

    # Normaliser les marginales
    marginal_H /= marginal_H.max()
    marginal_W /= marginal_W.max()

    # Créer la figure
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    # Afficher la heatmap avec un contour plot
    contour = axes[0, 1].contourf(likelihood_map_smoothed, levels=20, cmap="viridis")
    fig.colorbar(contour, ax=axes[0, 1], label='Probabilité de vraisemblance')
    axes[0, 1].set_title("Carte de probabilité de vraisemblance")

    # Afficher les marginales
    axes[1, 0].plot(marginal_H, range(len(pos_H)), color='blue')
    axes[1, 0].set_title("Probabilité marginale (axe vertical)")
    axes[1, 0].set_xlabel("Probabilité")
    axes[1, 0].set_ylabel("Position (H)")

    axes[1, 1].plot(range(len(pos_W)), marginal_W, color='red')
    axes[1, 1].set_title("Probabilité marginale (axe horizontal)")
    axes[1, 1].set_xlabel("Position (W)")
    axes[1, 1].set_ylabel("Probabilité")

    plt.tight_layout()
    return fig, axes

fig, axes = visualize_likelihood_map(results.mean(axis=-1), pos_H, pos_W)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import gaussian_filter

def visualize_likelihood_map(likelihood_map, pos_H, pos_W):
    # Lisser la heatmap avec un filtre gaussien
    # likelihood_map_smoothed = gaussian_filter(likelihood_map, sigma=3)
    likelihood_map_smoothed = likelihood_map

    # Calcul des probabilités marginales
    marginal_H = likelihood_map_smoothed.sum(axis=1)
    marginal_W = likelihood_map_smoothed.sum(axis=0)

    # Configurer le style avec seaborn pour un rendu plus esthétique
    sns.set(style="white", context="talk", font_scale=1.1)

    # Créer la figure
    fig, axes = plt.subplots(2, 2, figsize=(16, 14))

    # Afficher la heatmap avec contour plot
    contour = axes[0, 1].contourf(likelihood_map_smoothed, levels=20, cmap="viridis")
    fig.colorbar(contour, ax=axes[0, 1], label='Probabilité de vraisemblance')
    axes[0, 1].set_title("Carte de probabilité de vraisemblance", fontsize=14)
    axes[0, 1].set_xticks([])
    axes[0, 1].set_yticks([])

    # Ajouter des lignes de contour pour plus de clarté
    axes[0, 1].contour(likelihood_map_smoothed, levels=10, colors='white', alpha=0.5, linewidths=1)

    # Afficher les marginales avec des améliorations esthétiques
    # sns.lineplot(x=marginal_H, y=range(len(pos_H)), ax=axes[1, 0], color='darkblue', linewidth=2.5)
    # axes[1, 0].set_title("Probabilité marginale (axe vertical)", fontsize=14)
    # axes[1, 0].set_xlabel("Probabilité", fontsize=12)
    # axes[1, 0].set_ylabel("Position (H)", fontsize=12)
    # axes[1, 0].grid(True, linestyle='--', alpha=0.6)

    # sns.lineplot(x=range(len(pos_W)), y=marginal_W, ax=axes[1, 1], color='darkred', linewidth=2.5)
    # axes[1, 1].set_title("Probabilité marginale (axe horizontal)", fontsize=14)
    # axes[1, 1].set_xlabel("Position (W)", fontsize=12)
    # axes[1, 1].set_ylabel("Probabilité", fontsize=12)
    # axes[1, 1].grid(True, linestyle='--', alpha=0.6)

    # Ajuster la mise en page
    plt.tight_layout()
    sns.despine()

    return fig, axes

fig, axes = visualize_likelihood_map(results.mean(axis=-1), pos_H, pos_W)


In [ ]:
accuracy = results.max(axis=(0, 1))
fig, ax = fovea.plt.subplots()
ax.hist(accuracy)
